# Heat Budget

Hedemann et al. (2016): the 100 m OHC should correlate very well with GMST

the heat budget of that layer can then be traced to (TAO + atm. uptake) and (deep ocean OHC uptake).

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import path_samoc, path_results
from constants import spy, A_earth, Jpy_to_Wpsm

In [ ]:
from paths import file_ex_ocn_ctrl
from constants import cp_sw

In [ ]:
# top of atmosphere (really, top of model) radiative imbalance in [W]
toa_ctrl = xr.open_dataarray(f'{path_results}/TOA/TOM_ctrl.nc')
toa_rcp  = xr.open_dataarray(f'{path_results}/TOA/TOM_rcp.nc' )

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Global_Ocean.nc'  , decode_times=False)
ctrl_A = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Atlantic_Ocean.nc', decode_times=False)
ctrl_P = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Pacific_Ocean.nc' , decode_times=False)
ctrl_I = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Indian_Ocean.nc'  , decode_times=False)
ctrl_M = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Mediterranean.nc' , decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl_Southern_Ocean.nc', decode_times=False)
rcp    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Global_Ocean.nc'   , decode_times=False)
rcp_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Atlantic_Ocean.nc' , decode_times=False)
rcp_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Pacific_Ocean.nc'  , decode_times=False)
rcp_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Indian_Ocean.nc'   , decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Mediterranean.nc'  , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp_Southern_Ocean.nc' , decode_times=False)

In [ ]:
# something is still wrong in CTRL year 205
for ds in [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S]:
    for field in ['OHC_global', 'OHC_global_levels', 'OHC_zonal', 'OHC_zonal_levels']:
        ds[field][105] = (ds[field].sel({'time':204*365}) +
                                           ds[field].sel({'time':206*365}) )/2

In [ ]:
OHC_upper_100m_ctrl = ctrl.OHC_global_levels.isel(z_t=slice(0, 9)).sum(dim=('z_t'))
OHC_upper_100m_rcp  = rcp .OHC_global_levels.isel(z_t=slice(0, 9)).sum(dim=('z_t'))
OHC_below_100m_ctrl = ctrl.OHC_global_levels.isel(z_t=slice(9,42)).sum(dim=('z_t'))
OHC_below_100m_rcp  = rcp .OHC_global_levels.isel(z_t=slice(9,42)).sum(dim=('z_t'))

In [ ]:
f,ax = plt.subplots(1,2,figsize=(12,5))
(OHC_upper_100m_ctrl-OHC_upper_100m_ctrl.mean(dim='time')).plot(ax=ax[0])
(OHC_below_100m_ctrl-OHC_below_100m_ctrl.mean(dim='time')).plot(ax=ax[0])
(OHC_upper_100m_rcp -OHC_upper_100m_rcp.mean( dim='time')).plot(ax=ax[1])
(OHC_below_100m_rcp -OHC_below_100m_rcp.mean( dim='time')).plot(ax=ax[1])
plt.tight_layout()

In [ ]:
# differencing: chosen b/c physical interpretation as year-on-year heat difference
# (and fitting linear/quad./exp. model does not seem obvious in rcp)

OHC_upper_100m_diff_ctrl = (OHC_upper_100m_ctrl-OHC_upper_100m_ctrl.shift(time=1)).dropna(dim='time')
OHC_upper_100m_diff_rcp  = (OHC_upper_100m_rcp -OHC_upper_100m_rcp.shift(time=1) ).dropna(dim='time')
OHC_below_100m_diff_ctrl = (OHC_below_100m_ctrl-OHC_below_100m_ctrl.shift(time=1)).dropna(dim='time')
OHC_below_100m_diff_rcp  = (OHC_below_100m_rcp -OHC_below_100m_rcp.shift(time=1) ).dropna(dim='time')

# check autocorrelation
def auto_corr(da):
    return np.corrcoef(da[1:], da.shift(time=1).dropna(dim='time'))[0,1]
print(auto_corr(OHC_upper_100m_diff_ctrl), auto_corr(OHC_upper_100m_diff_rcp),
      auto_corr(OHC_below_100m_diff_ctrl), auto_corr(OHC_below_100m_diff_rcp) )
# autocorrelation low, yearly data appears like independent data points

# linearly detrend the data
def lin_detr_ts(da):
    fit_par = np.polyfit(da.time, da.values, 1)
    detr = da - fit_par[0]*da.time - fit_par[1]
    return detr

OHC_upper_100m_diff_detr_ctrl = lin_detr_ts(OHC_upper_100m_diff_ctrl)
OHC_upper_100m_diff_detr_rcp  = lin_detr_ts(OHC_upper_100m_diff_rcp )
OHC_below_100m_diff_detr_ctrl = lin_detr_ts(OHC_below_100m_diff_ctrl)
OHC_below_100m_diff_detr_rcp  = lin_detr_ts(OHC_below_100m_diff_rcp )

toa_detr_ctrl = lin_detr_ts(toa_ctrl)
toa_detr_rcp  = lin_detr_ts(toa_rcp )

print(auto_corr(OHC_upper_100m_diff_detr_ctrl), auto_corr(OHC_upper_100m_diff_detr_rcp),
      auto_corr(OHC_below_100m_diff_detr_ctrl), auto_corr(OHC_below_100m_diff_detr_rcp) )
# barely any trend in ctrl: autocorr is still low
# in rcp trend correction lowers autocorr

In [ ]:
ctrl_A.time/365

In [ ]:
toa_ctrl

In [ ]:
f,ax = plt.subplots(2,2,figsize=(12,8), sharey='row', sharex='col')
ax[0,0].axhline(0, c='k', lw=.5)
ax[0,0].plot(ctrl.time[1:]/365, OHC_upper_100m_diff_ctrl/1e21, lw=.5, label='$\Delta$OHC upper 100 m')
ax[0,0].plot(ctrl.time[1:]/365, OHC_below_100m_diff_ctrl/1e21, lw=.5, label='$\Delta$OHC below 100 m')
ax[0,0].plot(ctrl.time/365, toa_ctrl*spy            /1e21, lw=.5, label='TOA imbalance'          )

ax[1,0].axhline(0, c='k', lw=.5)
ax[1,0].plot(ctrl.time/365, OHC_upper_100m_diff_detr_ctrl/1e21, lw=.5, label='$\Delta$OHC upper 100 m')
ax[1,0].plot(ctrl.time/365, OHC_below_100m_diff_detr_ctrl/1e21, lw=.5, label='$\Delta$OHC below 100 m')
ax[1,0].plot(ctrl.time/365, toa_detr_ctrl*spy            /1e21, lw=.5, label='TOA imbalance'          )

ax[0,0].text(.02,.92, 'CTRL', transform=ax[0,0].transAxes, fontsize=16)
ax[0,0].legend(fontsize=16, frameon=False)

ax[0,1].axhline(0, c='k', lw=.5)
ax[0,1].plot(rcp.time/365, OHC_upper_100m_diff_rcp/1e21, lw=.5, label='$\Delta$OHC upper 100 m')
ax[0,1].plot(rcp.time/365, OHC_below_100m_diff_rcp/1e21, lw=.5, label='$\Delta$OHC below 100 m')
ax[0,1].plot(rcp.time/365, toa_rcp *spy           /1e21, lw=.5, label='TOA imbalance')
ax[0,1].text(.02,.92, 'RCP', transform=ax[0,1].transAxes, fontsize=16)
# ax[0,1].legend(fontsize=16, frameon=False)

ax[1,1].axhline(0, c='k', lw=.5)
ax[1,1].plot(rcp.time/365, OHC_upper_100m_diff_detr_rcp/1e21, lw=.5, label='$\Delta$OHC upper 100 m')
ax[1,1].plot(rcp.time/365, OHC_below_100m_diff_detr_rcp/1e21, lw=.5, label='$\Delta$OHC below 100 m')
ax[1,1].plot(rcp.time/365, toa_detr_rcp *spy           /1e21, lw=.5, label='TOA imbalance')

for i in range(2):
    ax[i,0].set_ylabel('heat difference [ZJ/year]', fontsize=16)
    ax[1,i].set_xlabel('time [years]', fontsize=16)


plt.tight_layout()

we can remove linear trend as that is an artifact of the different heat capacities

In [ ]:
OHC_below_100m_diff_detr_ctrl

In [ ]:
toa_detr_ctrl

In [ ]:
# Hedemann plot
plt.figure(figsize=(8,5))

plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
plt.plot([-2,0], [-2,0], c='k', lw=.5, ls='--')
plt.plot([-2,2], [2,-2], c='k', lw=.5, ls='-')
plt.plot([-2,2], [2-.082,-2-.082], c='k', lw=.5, ls=':')
plt.scatter(OHC_below_100m_diff_detr_ctrl.rolling({'time':15},center=True).mean()*Jpy_to_Wpsm,
            toa_detr_ctrl[1:]            .rolling({'time':15},center=True).mean()/A_earth, c='C0')
plt.scatter(OHC_below_100m_diff_detr_rcp .rolling({'time':15},center=True).mean()*Jpy_to_Wpsm,
            toa_detr_rcp[1:-14]          .rolling({'time':15},center=True).mean()/A_earth, c='C1')
plt.ylim((-.4,.4))
plt.xlim((-.6,.6))
plt.xlabel('detrended ocean component [W m$^{-2}$]', fontsize=16)
plt.ylabel('TOA component [W m$^{-2}$]', fontsize=16)
plt.tight_layout()

what does "as 15-year ensemble anomalies" mean?

yearly anomalies to the 15 year mean or 15 yr averaged anamalies

In [ ]:
# differencing: chosen b/c physical interpretation as year-on-year heat difference
# (and fitting linear/quad./exp. model does not seem obvious in rcp)

f,ax = plt.subplots(1,2,figsize=(12,5), sharey=True)
ax[0].axhline(0, c='k', lw=.5)
# ax[0].plot(OHC_upper_100m_diff_ctrl/1e21, label='upper 100 m')
# ax[0].plot(OHC_below_100m_diff_ctrl/1e21, label='below 100 m')
# ax[0].plot(toa_diff_ctrl*spy       /1e21, label='TOA imbalance')
ax[0].plot(OHC_upper_100m_diff_ctrl.rolling({'time':5},center=True).mean()/1e21, label=f'$\Delta$OHC upper 100 m')
ax[0].plot(OHC_below_100m_diff_ctrl.rolling({'time':5},center=True).mean()/1e21, label=f'$\Delta$OHC below 100 m')
ax[0].plot(toa_ctrl.rolling({'time':5},center=True).mean()*spy            /1e21, label='TOA rad. imbalance')

# ax[0].text(.02,.93, f'corr. {corr_diff_ctrl:.3f}', transform=ax[0].transAxes, fontsize=16)
ax[0].text(.02,.02, 'CTRL 5 year running mean', transform=ax[0].transAxes, fontsize=16)
ax[0].legend(fontsize=16, frameon=False)

ax[1].axhline(0, c='k', lw=.5)
# ax[1].plot(OHC_upper_100m_diff_rcp/1e21, label='upper 100 m')
# ax[1].plot(OHC_below_100m_diff_rcp/1e21, label='below 100 m')
# ax[1].plot(toa_diff_rcp *spy      /1e21, label='TOA imbalance')
ax[1].plot(OHC_upper_100m_diff_rcp.rolling({'time':5},center=True).mean()/1e21, label=f'$\Delta$OHC upper 100 m')
ax[1].plot(OHC_below_100m_diff_rcp.rolling({'time':5},center=True).mean()/1e21, label=f'$\Delta$OHC below 100 m')
ax[1].plot(toa_rcp.rolling({'time':5},center=True).mean() *spy           /1e21, label='TOA rad. imbalance')

# ax[1].text(.02,.93, f'corr. {corr_diff_rcp:.3f}', transform=ax[1].transAxes, fontsize=16)
ax[1].text(.02,.02, 'RCP 5 year running mean', transform=ax[1].transAxes, fontsize=16)


ax[0].set_ylabel('heat difference [ZJ/year]', fontsize=16)
ax[1].legend(fontsize=16, frameon=False)
plt.tight_layout()



clearly, a lot of the exchange between the upper 100m and the ocean below is responsible for the variability of the OHC in the surface layer

In [ ]:
plt.plot(((OHC_100m_ctrl-OHC_100m_ctrl.mean(dim='time'))/OHC_100m_ctrl.std(dim='time')))
plt.plot(((gmst_ctrl.GMST-gmst_ctrl.GMST.mean(dim='year'))/gmst_ctrl.GMST.std(dim='year')))
np.corrcoef(OHC_100m_ctrl, gmst_ctrl.GMST)

In [ ]:
np.corrcoef(OHC_100m_rcp, gmst_rcp.GMST[:67])

In [ ]:
np.corrcoeff(OHC_100m_ctrl, gmst_ctrl)

In [ ]:
for i, z in enumerate(ctrl.OHC_zonal_levels.z_t.values):
    print(i,z)